In [40]:
from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline
plt.rcParams['figure.figsize']= (8,8)

from warnings import filterwarnings
filterwarnings('ignore')

## Load Data

In [41]:
train_features = pd.read_csv('./data/dengue_features_train.csv', index_col=[0,1,2])
train_labels = pd.read_csv('./data/dengue_labels_train.csv', index_col=[0,1,2])


## Preprocess Data
Preprocess data into features required for training.

In [42]:
def preprocess_data(train_features, train_labels=None):
    df = train_features.copy()
    
    features = ['station_max_temp_c', 
                 'station_min_temp_c', 
                 'station_avg_temp_c', 
                 'precipitation_amt_mm', 
                 'reanalysis_dew_point_temp_k', 
                 'reanalysis_air_temp_k', 
                 'reanalysis_relative_humidity_percent',
                 'reanalysis_precip_amt_kg_per_m2', 
                 'ndvi_se', 
                 'ndvi_ne', 
                 'station_precip_mm']
    
    df = df[features]
    df.fillna(method='ffill', inplace=True)

    
    if train_labels is not None:
        labels = train_labels
        df = df.join(labels)
        
   
    celsius_list = ['station_max_temp_c', 'station_min_temp_c', 'station_avg_temp_c' ]
    for col in celsius_list:
        df[col] = df[col] + 273.15
    df = df.rename(columns = { name : name[:-2] + '_k' for name in celsius_list })
    sj = df.loc['sj']
    iq = df.loc['iq']
    
    return sj, iq

## Utility Functions

In [43]:
def plot_against(sj_pred, iq_pred):
    figs, axes = plt.subplots(nrows=2, ncols=1)

    # plot sj
    sj_pred.plot(ax=axes[0], label="Predictions")
    sj_train.total_cases.plot(ax=axes[0], label="Actual")

    # plot iq
    iq_pred.plot(ax=axes[1], label="Predictions")
    iq_train.total_cases.plot(ax=axes[1], label="Actual")

    plt.suptitle("Dengue Predicted Cases vs. Actual Cases")
    plt.legend()

In [44]:
def mae_calc(test, predicted):
    return (np.abs(test - predicted).sum())/len(test)

In [45]:
def save_benchmark(sj_predictions, iq_predictions):
    submission = pd.read_csv('./data/submission_format.csv', index_col=[0,1,2])
    submission.total_cases = np.concatenate([sj_predictions, iq_predictions])
    submission.to_csv("benchmark.csv")

## Split Data
Split into Training and Testing data

In [46]:
sj_train, iq_train = preprocess_data(train_features, train_labels)

sj_train_subtrain = sj_train.head(800)
sj_train_subtest = sj_train.tail(sj_train.shape[0] - 800)

iq_train_subtrain = iq_train.head(400)
iq_train_subtest = iq_train.tail(iq_train.shape[0] - 400)

## Statsmodel GLM Model
GLM models with different features for San Juan and Iquitos

In [47]:
def sj_glm_model(train, test):
    model_formula = "total_cases ~ 1 + " \
                    "reanalysis_relative_humidity_percent + reanalysis_dew_point_temp_k + " \
                    "station_min_temp_k + station_avg_temp_k"
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)
    best_alpha = []
    best_score = 1000
    
    for alpha in grid:
        model = smf.glm(formula=model_formula, data=train, family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(test).astype(int)
        score = eval_measures.meanabs(predictions, test.total_cases)

        if score < best_score:
            best_alpha = alpha
            best_score = score

    print('best alpha = ', best_alpha)
    print('best score = ', best_score)
    
    full_dataset = pd.concat([train, test])
    model = smf.glm(formula=model_formula, data=full_dataset, family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model

In [48]:
def iq_glm_model(train, test):
    model_formula = "total_cases ~ 1 + " \
                    "station_min_temp_k + station_avg_temp_k + station_max_temp_k + " \
                    "precipitation_amt_mm + reanalysis_dew_point_temp_k + reanalysis_air_temp_k + " \
                    "reanalysis_relative_humidity_percent + reanalysis_precip_amt_kg_per_m2 + " \
                    "ndvi_se + ndvi_ne + station_precip_mm"
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)             
    best_alpha = []
    best_score = 1000
    
    for alpha in grid:
        model = smf.glm(formula=model_formula, data=train, family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(test).astype(int)
        score = eval_measures.meanabs(predictions, test.total_cases)

        if score < best_score:
            best_alpha = alpha
            best_score = score

    print('best alpha = ', best_alpha)
    print('best score = ', best_score)
    
    full_dataset = pd.concat([train, test])
    model = smf.glm(formula=model_formula, data=full_dataset, family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model    

In [49]:
# your code here
def poly_rfr_model(train, test):
    X = train.drop(columns=["total_cases"])
    y = train.total_cases

    depth = range(1, 11)
    n_estimator = [10, 30, 50, 80, 100, 300, 500, 800, 1000]

    polynomial_features = PolynomialFeatures(degree=2, include_bias=True)
    rfr = RandomForestRegressor(random_state=67)
    pipeline_rfr = Pipeline([("polynomial_features", polynomial_features), ("rfr", rfr)])

    parameters = [{"rfr__max_depth":depth, "rfr__n_estimators":n_estimator}]
    clf_rfr = GridSearchCV(pipeline_rfr, param_grid=parameters, cv=10, scoring='neg_mean_absolute_error')
    clf_rfr.fit(X, y)

    scores = clf_rfr.cv_results_['mean_test_score']
    scores_std = clf_rfr.cv_results_['std_test_score']
    std_error = scores_std / np.sqrt(n_folds)

    print("Best depth for Random Forest is", clf_rfr.best_params_['rfr__max_depth'])
    print("Best number of neighbours for Random Forest is", clf_rfr.best_params_['rfr__n_estimators'])

    polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
    rfr = RandomForestRegressor(random_state=67, max_depth=clf_rfr.best_params_['rfr__max_depth'], n_estimators=clf_rfr.best_params_['rfr__n_estimators'])
    pipeline_rfr = Pipeline([("polynomial_features", polynomial_features), ("rfr", rfr)])
    pipeline_rfr.fit(X, y)

In [50]:
def random_forest_model(train, test):
    X = train.drop(columns=['total_cases'])
    Y = train.total_cases
    
    reg = RandomForestRegressor(random_state=67)
    param_grid = [{
      'n_estimators': [10, 30, 100, 300, 500, 1000], 
      'max_depth': [3, 5, 7, None]
    }]
    grid = GridSearchCV(reg, param_grid=param_grid, scoring='neg_mean_absolute_error')
    grid.fit(X, Y)
    
    print("Best score: {}".format(np.abs(grid.best_score_)))
    print("Best params: {}".format(grid.best_params_))
    
    full_dataset = pd.concat([train, test])
    reg = RandomForestRegressor(random_state=67, max_depth=grid.best_params_['max_depth'], n_estimators=grid.best_params_['n_estimators'])
    reg.fit(X, Y)
    
    return reg

In [51]:
def poly_knn_model(train, test):
    X = train.drop(columns=['total_cases'])
    Y = train.total_cases

    k_neighbours = range(1, 26)

    polynomial_features = PolynomialFeatures(degree=2, include_bias=True)
    knn = KNeighborsRegressor()
    pipeline_knn = Pipeline([("polynomial_features", polynomial_features), ("knn", knn)])

    parameters = [{"knn__n_neighbors":k_neighbours}]

    clf_knn = GridSearchCV(pipeline_knn, param_grid=parameters, cv=10, scoring='neg_mean_squared_error')
    clf_knn.fit(X, Y)

    scores = clf_knn.cv_results_['mean_test_score']
    scores_std = clf_knn.cv_results_['std_test_score']
    std_error = scores_std / np.sqrt(n_folds)

    print("Best k for kNN is", clf_knn.best_params_['knn__n_neighbors'])

    polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
    knn = KNeighborsRegressor(n_neighbors=clf_knn.best_params_['knn__n_neighbors'])
    pipeline_knn = Pipeline([("polynomial_features", polynomial_features), ("knn", knn)])
    pipeline_knn.fit(X, Y)

In [52]:
def kneighbours_model(train, test):
    X = train.drop(columns=['total_cases'])
    Y = train.total_cases
    
    k_neighbours = range(1, 26)
    
    reg = KNeighborsRegressor(random_state=67)
    param_grid = [{"n_neighbors":k_neighbours}]
    grid = GridSearchCV(reg, param_grid=param_grid, scoring='neg_mean_absolute_error')
    grid.fit(X, Y)
    
    print("Best score: {}".format(np.abs(grid.best_score_)))
    print("Best params: {}".format(grid.best_params_))
    
    full_dataset = pd.concat([train, test])
    reg = KNeighborsRegressor(random_state=67, n_neighbors=grid.best_params_['n_neighbors'])
    reg.fit(X, Y)
    
    return reg

## Check Results
Checks results of models by plotting against actual values and calculating the Mean Absolute Error for the individual models and together.

In [53]:
models = ["GLM", "Poly RFR", "RFR", "Poly KNN", "KNN"]
sj_models_mae = []
iq_models_mae = []

In [54]:
sj_best_model = sj_glm_model(sj_train_subtrain, sj_train_subtest)
iq_best_model = iq_glm_model(iq_train_subtrain, iq_train_subtest)
pred_sj = sj_best_model.fittedvalues
pred_iq = iq_best_model.fittedvalues

plot_against(pred_sj, iq_train.fitted)

sj_mae = mae_calc(pred_sj, sj_train.total_cases)
sj_models_mae.append(sj_mae)
print("San Juan MAE:", sj_mae)
iq_mae = mae_calc(pred_iq, iq_train.total_cases)
iq_models_mae.append(iq_mae)
print("Iquitos MAE:", iq_mae)
total_mae = (np.abs(pred_sj - sj_train.total_cases).sum() + 
             np.abs(pred_iq - iq_train.total_cases).sum()) / (sj_train.shape[0] + iq_train.shape[0])
print("Total MAE:", total_mae)

best alpha =  1e-08
best score =  21.698529411764707
best alpha =  1e-08
best score =  6.991666666666666


AttributeError: 'DataFrame' object has no attribute 'fitted'

In [ ]:
sj_best_model = poly_rfr_model(sj_train_subtrain, sj_train_subtest)
iq_best_model = poly_rfr_model(iq_train_subtrain, iq_train_subtest)
pred_sj = sj_best_model.predict(sj_train.drop(columns=["total_cases"]))
pred_iq = iq_best_model.predict(iq_train.drop(columns=["total_cases"]))

plot_against(pred_sj, iq_train.fitted)

sj_mae = mae_calc(pred_sj, sj_train.total_cases)
sj_models_mae.append(sj_mae)
print("San Juan MAE:", sj_mae)
iq_mae = mae_calc(pred_iq, iq_train.total_cases)
iq_models_mae.append(iq_mae)
print("Iquitos MAE:", iq_mae)
total_mae = (np.abs(pred_sj - sj_train.total_cases).sum() + 
             np.abs(pred_iq - iq_train.total_cases).sum()) / (sj_train.shape[0] + iq_train.shape[0])
print("Total MAE:", total_mae)

In [ ]:
sj_best_model = random_forest_model(sj_train_subtrain, sj_train_subtest)
iq_best_model = random_forest_model(iq_train_subtrain, iq_train_subtest)
pred_sj = sj_best_model.predict(sj_train.drop(columns=["total_cases"]))
pred_iq = iq_best_model.predict(iq_train.drop(columns=["total_cases"]))

plot_against(pred_sj, iq_train.fitted)

sj_mae = mae_calc(pred_sj, sj_train.total_cases)
sj_models_mae.append(sj_mae)
print("San Juan MAE:", sj_mae)
iq_mae = mae_calc(pred_iq, iq_train.total_cases)
iq_models_mae.append(iq_mae)
print("Iquitos MAE:", iq_mae)
total_mae = (np.abs(pred_sj - sj_train.total_cases).sum() + 
             np.abs(pred_iq - iq_train.total_cases).sum()) / (sj_train.shape[0] + iq_train.shape[0])
print("Total MAE:", total_mae)

In [ ]:
sj_best_model = poly_knn_model(sj_train_subtrain, sj_train_subtest)
iq_best_model = poly_knn_model(iq_train_subtrain, iq_train_subtest)
pred_sj = sj_best_model.predict(sj_train.drop(columns=["total_cases"]))
pred_iq = iq_best_model.predict(iq_train.drop(columns=["total_cases"]))

plot_against(pred_sj, iq_train.fitted)

sj_mae = mae_calc(pred_sj, sj_train.total_cases)
sj_models_mae.append(sj_mae)
print("San Juan MAE:", sj_mae)
iq_mae = mae_calc(pred_iq, iq_train.total_cases)
iq_models_mae.append(iq_mae)
print("Iquitos MAE:", iq_mae)
total_mae = (np.abs(pred_sj - sj_train.total_cases).sum() + 
             np.abs(pred_iq - iq_train.total_cases).sum()) / (sj_train.shape[0] + iq_train.shape[0])
print("Total MAE:", total_mae)

In [ ]:
sj_best_model = kneighbours_model(sj_train_subtrain, sj_train_subtest)
iq_best_model = kneighbours_model(iq_train_subtrain, iq_train_subtest)
pred_sj = sj_best_model.predict(sj_train.drop(columns=["total_cases"]))
pred_iq = iq_best_model.predict(iq_train.drop(columns=["total_cases"]))

plot_against(pred_sj, iq_train.fitted)

sj_mae = mae_calc(pred_sj, sj_train.total_cases)
sj_models_mae.append(sj_mae)
print("San Juan MAE:", sj_mae)
iq_mae = mae_calc(pred_iq, iq_train.total_cases)
iq_models_mae.append(iq_mae)
print("Iquitos MAE:", iq_mae)
total_mae = (np.abs(pred_sj - sj_train.total_cases).sum() + 
             np.abs(pred_iq - iq_train.total_cases).sum()) / (sj_train.shape[0] + iq_train.shape[0])
print("Total MAE:", total_mae)

In [ ]:
error_map = pd.DataFrame()
error_map["models"] = models
error_map["sj_mae"] = sj_models_mae
error_map["iq_mae"] = iq_models_mae
sns.barplot(x=error_map.models, y=error_map.sj_mae)
plt.show()
sns.barplot(x=error_map.models, y=error_map.iq_mae)

## Predict on test set
Predicts on the test set given and saves predictions to benchmark.csv using the submission format given.

In [39]:
test_features = pd.read_csv('./data/dengue_features_test.csv', index_col=[0,1,2])
sj_test, iq_test = preprocess_data(test_features)

sj_predictions = sj_best_model.predict(sj_test).astype(int)
iq_predictions = iq_best_model.predict(iq_test).astype(int)
save_benchmark(sj_predictions, iq_predictions)